Expected Value, Win Probability, and why "Common Knowledge" Hurts Sports Teams

Tutorial by Chris Berry

The four major American sports leagues (MLB, NFL, NBA, and NHL) have very long and extensive histories, and established throughout each league's history were traditions and "common knowledge" that many managers and front offices adhere to, to this day. However, many of the rules and practices that teams follow simply do not match up with what they mathematically <i>should</i> be doing. 

The goal of every sports team should be to win games, and so every decision made by a coach should be made with the intention of increasing the probability that that team will win the game. For example, if a football coach thinks that kicking an onside kick will make them more likely to win the game, then they should choose to kick an onside kick. Despite this, there are many decisions that coaches make that are made because of deep-rooted traditions in the sport, without considering the data whatsoever. One of the most famous examples of this is punting in football: conventional wisdom states that football teams should almost never go for it on 4th down, but many different sources show that punting is almost never the mathematically correct decision (https://www.footballstudyhall.com/2013/11/15/5105958/fourth-down-pulaski-academy-kevin-kelley#:~:text=Kevin%20Kelley%2C%20Pulaski's%20coach%20never,attempts%20onside%20kicks%20after%20scoring). 

In this tutorial I will discuss the ways in which win probability is used and misused in the MLB, NFL, NBA, and NHL. In doing so, I will walk through all the steps of analyzing data, from retreiving, cleaning, and processing the data to creating predictive models of different real-life scenareos.

Part 1: Expected Value and the 2015-16 Golden State Warriors

Before we talk about win probability, we need to talk about a concept called "expected value". In the context of sports, expected value (EV) is a measure of how many points that you can expect will result from a decision. The equation for EV is (equation). For example, say we are playing a game, where whenever I flip a coin, if it lands on heads, I give you $10, and if it lands on tails, you give me $5. The equation for EV in this situation is EV = .5*($10) + .5*(-$5) = $2.5; therefore, you can "expect" to win $2.50 every time I flip the coin. 

As another example, take Steve Nash: in the 2005-06 season, Nash made 54.8\% of his 2-point shots, and 43.9\% of his 3-point shots. For that season, his expected value of making a 2-point shot was .548*(2) = 1.096 points, and his expected value of making a 3-point shot was .439*(3) = 1.317 points. This means that, with everything else being equal, a 3-point attempt from Nash was worth about 0.221 points more valuable than a 2-point attempt. Despite this, he attempted 9.0 2-point shots per game that season, while only attempting 4.3 3-point shots per game. If a 3-point shot was worth more than a 2-point shot, why didn't he shoot more 3's? One answer is because, in the 2000s, the 3-pointer was considered a bad shot. Many players and coaches believed that it was a losing strategy to shoot more 3-pointers than 2-pointers, in part because that's not a strategy that had ever worked well enough to win a team a championship. That all changed after Stephen Curry proved just how effective 3's could be.

In the 2015-16 season, Steph shot 45.4% from 3-point range and 56.6% from 2 point range, meaning that his EV for 2-pointers and 3-pointers were 1.132 and 1.362 points per shot, respectively. However, Curry had 11.2 3-point attempts per game, compared to 9.0 2-point attempts. This was one reason why his Golden State Warriors won a record 73 games that season, and one reason why he recieved every single first-place MVP vote that season. This shows that expected value plays a major role in the success of a sports team.

While choosing the higher EV decision doesn't always lead to a higher win probability (more on that later), due to basketball's high scoring volume and high number of possessions per game, always taking high EV shots will almost always lead to a team winning. Steph's 2015-16 MVP season was a wake-up call for the entire NBA; would shooting higher-EV shots lead to more success?

In this first part, we will determine how predictive taking high-EV shots 

In [471]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
from matplotlib import pyplot as plt
import sqlite3
import statistics
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR

pref = "D:\\Documents\\Computer Programs\\CMSC320\\Final_Project\\"

In [7]:
data = pd.read_csv(pref + "NBA\\2016-17_NBA_Shot_data.csv")
data.head()
# calculate each player's proportion of high EV shots, and whether that relates to team success

,Rk,Player▲,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines\abrinal01,SG,23,OKC,68,6,15.5,2.0,5.0,...,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0
1,2,Quincy Acy\acyqu01,PF,26,TOT,38,1,14.7,1.8,4.5,...,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8
2,3,Steven Adams\adamsst01,C,23,OKC,80,80,29.9,4.7,8.2,...,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3
3,4,Arron Afflalo\afflaar01,SG,31,SAC,61,45,25.9,3.0,6.9,...,0.892,0.1,1.9,2.0,1.3,0.3,0.1,0.7,1.7,8.4
4,5,Alexis Ajinça\ajincal01,C,28,NOP,39,15,15.0,2.3,4.6,...,0.725,1.2,3.4,4.5,0.3,0.5,0.6,0.8,2.0,5.3


Part 2: Pulling the Goalie

As stated above, basketball is a game where playing with EV in mind is important in just about every single minute of the game. In ice hockey, that isn't necessarily the case.

Imagine a hockey game between the Boston Bruins and the New York Rangers: there are  2 minutes left in the game, and the Bruins are losing 2-3. The Bruins decide to substitute their goalie for another skater. Andrew Thomas of Harvard University analyzed data from all NHL games between 2003 and 2007 and found that, in games where a goalie was pulled, the winning team scored on the empty net 34% of the time, while the team with the extra skater scored 30% of the time (https://fliphtml5.com/snvq/xptx/basic). So if the Rangers and Bruins are evenly matched, then the Bruins' expected value of pulling the goalie is EV = .3 * (1) + .34 * (-1) = -.04. 

If the Bruins get negative EV from pulling the goalie, then why do they do it? The answer is because they don't care whether they lose by 1 or lose by 2: they just want to tie the game. Despite being a negative EV decision, deciding to pull the goalie increases the Bruins' chances of winning the game. It would be terribly stupid to pull the goalie while the score is still 0-0 in the first period, but it becomes the smart thing to do if you're losing in the third. This begs the question: when should a team decide to pull their goalie?

In this part, we will be using NHL Game Data courtesy of Martin Ellis (https://www.kaggle.com/martinellis/nhl-game-data?select=game_plays.csv). This database contains game data from almost all NHL games played between the 2000-01 season and the 2019-20 season. We will be extracting data from 4 of the tables in this database: "total_shifts" tells us how much time each player is on the ice; "total_plays" tells us about every single play that happens in each game, be it a shot, penalty, goal, faceoff, or anything else; "games" gives us general stats about each game; and "game_goalie_stats" gives us the stats for each goaltender for every game. The ultimate goal is to learn more about when the optimal time to pull the goalie is in a losing game, based on available data.

We start by importing the data into Python. The "drop_duplicates()" command ensures that no game, goal, or player is being double-counted.

In [205]:
total_shifts = pd.read_csv(pref + "NHL\\game_shifts.csv")
total_shifts = total_shifts.drop_duplicates()
total_plays = pd.read_csv(pref + "NHL\\game_plays.csv")
total_plays = total_plays.drop_duplicates()
games = pd.read_csv(pref + "NHL\\game.csv")
games = games.drop_duplicates(subset=["game_id"])
game_goalie_stats = pd.read_csv(pref + "NHL\\game_goalie_stats.csv")
game_goalie_stats = game_goalie_stats.drop_duplicates()

Next, we will find every game in which a team was losing at the end of the game and decided to pull the goalie. Unfortunately, this information is not readily available in the database, so we will have to extrapolate. So in this step, we will first isolate every goalie's shift. Goalies are the only players in hockey who stay on the ice for more than just a minute or so at a time. All goalies start their shift at the beginning of the third period (2400 seconds into the game), and given that the longest shift in NHL history is around 310 seconds, we can safely say that if a shift starts at time marker 2400 and ends beyond 2710, then it's a goalie's shift.

We also know that if a goalie finished the game in net (i.e. their shift ends at time marker 3600), then they were not pulled. Therefore, any goalie shift that ends before the 3600 mark must have been pulled. We store these shifts in the "pulled_goalies" variable.

In [321]:
goalies = total_shifts[ (total_shifts["shift_start"] == 2400) & (total_shifts["shift_end"] > 2710) ]
pulled_goalies = goalies[goalies["shift_end"] < 3600]


There are almost always only 2 goalies playing per game, which means that by dividing the total number of third-period goalie shifts by two, we will get the total number of games in this data set. Dividing the number of games with a pulled goalie by the total number of games yields the proportion of games that have a pulled goalie:

In [322]:
number_of_games = goalies.shape[0] / 2
print(pulled_goalies.shape[0] / number_of_games)


0.7292955290796629


So about 72.93% of games have a situation where the goalie is pulled at the end of the game. (SAY PROBLEMS WITH THIS NUMBER)

In [323]:
pulled_goalies.head()

,game_id,player_id,period,shift_start,shift_end
1599,2018020003,8469608,3,2400,3469.0
2646,2018020004,8474889,3,2400,3511.0
6597,2018020005,8475622,3,2400,3297.0
8294,2018020008,8475852,3,2400,3418.0
8708,2018020006,8468685,3,2400,3489.0


This is a small section of the "pulled_goalies" table. From now on, we will use this table to represent the games that have a pulled goalie in them. This table doesn't tell us very much about each game, so we'll use the dataset to add more and more useful information. First, the season that each game was played in, as well as the time left in the game when the goalie was pulled:

In [333]:
seasons = []
for index, row in pulled_goalies.iterrows():
    seasons.append(int(games.loc[games["game_id"] == row["game_id"], "season"].drop_duplicates()))
pulled_goalies["season"] = seasons

pulled_goalies["time_left"] = 3600 - pulled_goalies["shift_end"]

pulled_goalies.head()

<ipython-input-333-fd82a71ee41f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pulled_goalies["season"] = seasons
<ipython-input-333-fd82a71ee41f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pulled_goalies["time_left"] = 3600 - pulled_goalies["shift_end"]


,game_id,player_id,period,shift_start,shift_end,season,time_left
1599,2018020003,8469608,3,2400,3469.0,20182019,131.0
2646,2018020004,8474889,3,2400,3511.0,20182019,89.0
6597,2018020005,8475622,3,2400,3297.0,20182019,303.0
8294,2018020008,8475852,3,2400,3418.0,20182019,182.0
8708,2018020006,8468685,3,2400,3489.0,20182019,111.0


Next, we'll figure out what teams played in this game, what the score was when the goalie was pulled, and what happened afterwards. To do this, we'll start by creating another table containing every goal, as well as what game the goal belongs to:

In [334]:
goals = total_plays[total_plays["event"] == "Goal"]
goals["gameTime"] = goals["periodTime"] + (goals["period"] - 1) * 1200

<ipython-input-334-87214652f07c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goals["gameTime"] = goals["periodTime"] + (goals["period"] - 1) * 1200


This next chunk of code looks intimidating, but it's simpler than it looks. We want a few bits of information from the "goals" table: for each game we want to figure out who's winning, who's losing, what's the score, how much is the losing team losing by, how many goals did the losing team get after pulling the goalie, and did the winning team score at all.

In [335]:

losing_scores = []
losing_teams = []
winning_scores = []
winning_teams = []
goals_for_losing_team = []
did_winning_team_score = []
for index, row in pulled_goalies.iterrows():
    game = row["game_id"]
    time = row["shift_end"]
    home_team = games[games["game_id"] == game]
    home_team = int(home_team["home_team_id"])
    away_team = games[games["game_id"] == game]
    away_team = int(away_team["away_team_id"])
    p3_goals = goals[goals["game_id"] == game]
    post_hook_goals = p3_goals[(p3_goals["gameTime"] > time)&(p3_goals["period"] == 3)]
    
    p3_goals = p3_goals[p3_goals["gameTime"] < time]
    
    if p3_goals.shape[0] > 0:
        
    
        goal_before = p3_goals[p3_goals["gameTime"] == p3_goals["gameTime"].max()]
        
        
        a = int(goal_before["goals_away"])
        h = int(goal_before["goals_home"])
        losing_score = ([a, away_team] if a < h else [h, home_team])
        winning_score = ([h, home_team] if a < h else [a, away_team])
        
        diff = winning_score[0] - losing_score[0]
        phg = 0 #goals scored by losing team before 1) the period ends, 2) the other team scores, or 3) they even the score
        dwts = 0
        if(len(post_hook_goals) > 0):
            g1 = post_hook_goals.iloc[0]
            if g1.loc["team_id_for"] == losing_score[1]:
                phg += 1
                if diff >= 2 and len(post_hook_goals) >= 2:
                    g2 = post_hook_goals.iloc[1]
                    if g2.loc["team_id_for"] == losing_score[1]:
                        phg += 1
                        if diff >= 3 and len(post_hook_goals) >= 3:
                            g3 = post_hook_goals.iloc[2]
                            if g3.loc["team_id_for"] == losing_score[1]:
                                phg += 1
                            else:
                                dwts = 1
                    else:
                        dwts = 1
            else:
                dwts = 1
        
        losing_scores.append(losing_score[0])
        losing_teams.append(losing_score[1])
        winning_scores.append(winning_score[0])
        winning_teams.append(winning_score[1])
        goals_for_losing_team.append(phg)
        did_winning_team_score.append(dwts)
    else:
        losing_scores.append(0)
        losing_teams.append(home_team)
        winning_scores.append(0)
        winning_teams.append(away_team)
        goals_for_losing_team.append(0)
        did_winning_team_score.append(0)
    
pulled_goalies["losingScore"] = losing_scores
pulled_goalies["losingTeam"] = losing_teams
pulled_goalies["winningScore"] = winning_scores
pulled_goalies["winningTeam"] = winning_teams
pulled_goalies["goalsForLosingTeam"] = goals_for_losing_team
pulled_goalies["didWinningTeamScore"] = did_winning_team_score

pulled_goalies = pulled_goalies[pulled_goalies["losingScore"] != pulled_goalies["winningScore"]]

pulled_goalies["goal_diff"] = pulled_goalies["winningScore"] - pulled_goalies["losingScore"]


pulled_goalies.head()
    
    


<ipython-input-335-7b9d1c5cfaf4>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pulled_goalies["losingScore"] = losing_scores
<ipython-input-335-7b9d1c5cfaf4>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pulled_goalies["losingTeam"] = losing_teams
<ipython-input-335-7b9d1c5cfaf4>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,game_id,player_id,period,shift_start,shift_end,season,time_left,losingScore,losingTeam,winningScore,winningTeam,goalsForLosingTeam,didWinningTeamScore,goal_diff
1599,2018020003,8469608,3,2400,3469.0,20182019,131.0,2,20,4,23,0,1,2
2646,2018020004,8474889,3,2400,3511.0,20182019,89.0,2,28,4,24,0,1,2
6597,2018020005,8475622,3,2400,3297.0,20182019,303.0,0,7,3,6,0,1,3
8294,2018020008,8475852,3,2400,3418.0,20182019,182.0,0,12,1,2,1,0,1
8708,2018020006,8468685,3,2400,3489.0,20182019,111.0,1,3,2,18,0,1,1


At this point, we have enough data to do some simple analysis. But what other factors might be able to predict the losing team's ability to score? Well, if the losing team is usually pretty good on offense, then clearly they will be more likely to come back. Likewise, if the winning team is usually pretty good on defense, then they will be less likely to come back. Because of this, we should calculate how good each losing team's offense is, as well as how good each winning team's defense is. How can we quantify this? One way of quantifying a team's offense is by using their Goals Per Game (GPG), which is the average number of goals that team scores in a game on average. A way of quantifying a team's defense is by using their goalie's Goals Against Average (GAA), which is the average number of goals that they allow per game on average.

In [336]:
seasons = []
for index, row in game_goalie_stats.iterrows():
    seasons.append(int(games.loc[games["game_id"] == row["game_id"], "season"].drop_duplicates()))
    

game_goalie_stats["season"] = seasons
goalies = game_goalie_stats[["team_id", "season"]].drop_duplicates()


game_goalie_stats["goalsAgainst"] = game_goalie_stats["shots"] - game_goalie_stats["saves"]
GAA = []
for index, row in goalies.iterrows():
    total_GA = game_goalie_stats.loc[(game_goalie_stats["team_id"] == row["team_id"])&(game_goalie_stats["season"] == row["season"]), "goalsAgainst"].sum()
    TOI = game_goalie_stats.loc[(game_goalie_stats["team_id"] == row["team_id"])&(game_goalie_stats["season"] == row["season"]), "timeOnIce"].sum()
    GAA.append(total_GA/(TOI/3600))
    

teams = games[["away_team_id", "season"]].drop_duplicates()
GPG = []
for index, row in teams.iterrows():
    total_GA = games.loc[(games["away_team_id"] == row["away_team_id"])&(games["season"] == row["season"]), "away_goals"].sum()
    total_GA += games.loc[(games["home_team_id"] == row["away_team_id"])&(games["season"] == row["season"]), "home_goals"].sum()
    total_games = int(games.loc[(games["away_team_id"] == row["away_team_id"])&(games["season"] == row["season"])].shape[0]*2)
    GPG.append(total_GA/total_games)
    
goalies["GAA"] = GAA
teams["GPG"] = GPG



We then add this information to the main table:

In [337]:
losing_GPG = []
winning_GAA = []
for index, row in pulled_goalies.iterrows():
    losing_GPG.append(float(teams.loc[(teams["away_team_id"] == row["losingTeam"])&(teams["season"] == row["season"]), "GPG"]))
    winning_GAA.append(float(goalies.loc[(goalies["team_id"] == row["winningTeam"])&(goalies["season"] == row["season"]), "GAA"]))
    
pulled_goalies["losingGPG"] = losing_GPG
pulled_goalies["winningGAA"] = winning_GAA

pulled_goalies.head()

,game_id,player_id,period,shift_start,shift_end,season,time_left,losingScore,losingTeam,winningScore,winningTeam,goalsForLosingTeam,didWinningTeamScore,goal_diff,losingGPG,winningGAA
1599,2018020003,8469608,3,2400,3469.0,20182019,131.0,2,20,4,23,0,1,2,3.488372,2.908813
2646,2018020004,8474889,3,2400,3511.0,20182019,89.0,2,28,4,24,0,1,2,3.470000,2.843036
6597,2018020005,8475622,3,2400,3297.0,20182019,303.0,0,7,3,6,0,1,3,2.756098,2.324237
8294,2018020008,8475852,3,2400,3418.0,20182019,182.0,0,12,1,2,1,0,1,2.897959,2.185846
8708,2018020006,8468685,3,2400,3489.0,20182019,111.0,1,3,2,18,0,1,1,2.768293,2.517451


In [338]:
pulled_goalies.head(30)

,game_id,player_id,period,shift_start,shift_end,season,time_left,losingScore,losingTeam,winningScore,winningTeam,goalsForLosingTeam,didWinningTeamScore,goal_diff,losingGPG,winningGAA
1599,2018020003,8469608,3,2400,3469.0,20182019,131.0,2,20,4,23,0,1,2,3.488372,2.908813
2646,2018020004,8474889,3,2400,3511.0,20182019,89.0,2,28,4,24,0,1,2,3.470000,2.843036
6597,2018020005,8475622,3,2400,3297.0,20182019,303.0,0,7,3,6,0,1,3,2.756098,2.324237
8294,2018020008,8475852,3,2400,3418.0,20182019,182.0,0,12,1,2,1,0,1,2.897959,2.185846
8708,2018020006,8468685,3,2400,3489.0,20182019,111.0,1,3,2,18,0,1,1,2.768293,2.517451
12941,2018020013,8471227,3,2400,3479.0,20182019,121.0,1,30,2,21,0,1,1,2.573171,2.706575
25661,2018020020,8475660,3,2400,3466.0,20182019,134.0,2,22,4,1,0,1,2,2.829268,3.086509
26968,2018020017,8468685,3,2400,2748.0,20182019,852.0,0,3,2,7,1,1,2,2.768293,3.047524
28082,2018020018,8475883,3,2400,3406.0,20182019,194.0,3,10,4,9,0,1,1,3.366667,3.476002
29407,2018020021,8471306,3,2400,3487.0,20182019,113.0,3,2,4,18,0,0,1,2.744444,2.517451


We finally have all the information we need. In our analysis, we'll define a few measures of success. The first is simply whether the losing team came back and won the game: 1 represents a win, 0 represents a loss.

In [342]:
did_they_win = []
for index, row in pulled_goalies.iterrows():
    succ = 0
    game = games.loc[games["game_id"] == row["game_id"]]
    
    if (row["losingTeam"] == game["away_team_id"]).bool() and ("away win" in str(game["outcome"])):
        succ = 1
    elif (row["losingTeam"] == game["home_team_id"]).bool() and "home win" in game["outcome"]:
        succ = 1
    did_they_win.append(succ)

pulled_goalies["didTheyWin"] = did_they_win


#pulled_goalies.head(20)

The next is whether the losing team overcame their defecit, regardless of whether they won or lost: 1 represents if they did, 0 represents if they didn't.

In [351]:
pulled_goalies["didTheyComeBack"] = np.where(pulled_goalies["goal_diff"] - pulled_goalies["goalsForLosingTeam"] == 0, 1, 0)
pulled_goalies.head(20)

,game_id,player_id,period,shift_start,shift_end,season,time_left,losingScore,losingTeam,winningScore,winningTeam,goalsForLosingTeam,didWinningTeamScore,goal_diff,losingGPG,winningGAA,success,didTheyWin,did_they_come_back,didTheyComeBack
1599,2018020003,8469608,3,2400,3469.0,20182019,131.0,2,20,4,23,0,1,2,3.488372,2.908813,0,0,0,0
2646,2018020004,8474889,3,2400,3511.0,20182019,89.0,2,28,4,24,0,1,2,3.470000,2.843036,0,0,0,0
6597,2018020005,8475622,3,2400,3297.0,20182019,303.0,0,7,3,6,0,1,3,2.756098,2.324237,0,0,0,0
8294,2018020008,8475852,3,2400,3418.0,20182019,182.0,0,12,1,2,1,0,1,2.897959,2.185846,0,0,1,1
8708,2018020006,8468685,3,2400,3489.0,20182019,111.0,1,3,2,18,0,1,1,2.768293,2.517451,0,0,0,0
12941,2018020013,8471227,3,2400,3479.0,20182019,121.0,1,30,2,21,0,1,1,2.573171,2.706575,0,0,0,0
25661,2018020020,8475660,3,2400,3466.0,20182019,134.0,2,22,4,1,0,1,2,2.829268,3.086509,0,0,0,0
26968,2018020017,8468685,3,2400,2748.0,20182019,852.0,0,3,2,7,1,1,2,2.768293,3.047524,0,0,0,0
28082,2018020018,8475883,3,2400,3406.0,20182019,194.0,3,10,4,9,0,1,1,3.366667,3.476002,0,0,0,0
29407,2018020021,8471306,3,2400,3487.0,20182019,113.0,3,2,4,18,0,0,1,2.744444,2.517451,0,0,0,0


The third measure of success will be net goals, i.e. the goals they scored with the net empty minus the goals their opponent scored.

In [433]:
pulled_goalies["net_goals"] = pulled_goalies["goalsForLosingTeam"] - pulled_goalies["didWinningTeamScore"]

Before we construct any models of this data, we will first make some general observations. First, one might wonder: how often are teams able to eliminate their defecit when they pull the goalie?

In [375]:
temp = pulled_goalies

print(temp.shape[0])

print(temp.loc[temp["didTheyComeBack"] == 1].shape[0])
print(temp.loc[temp["didTheyComeBack"] == 0].shape[0])
print(temp.loc[temp["didTheyComeBack"] == 1].shape[0]/temp.shape[0])

8182
761
7421
0.09300904424346125


According to our data, pulling the goalie works about 9.3% of the time. Now, assuming parity in the league, one would expect teams to win even-scored games about equally as often as they lose them, so one would expect the losing team to win about 4.65% of the time:

In [376]:
print(temp.loc[temp["didTheyWin"] == 1].shape[0]/temp.shape[0])

0.02309948667807382


However, this is not the case; the losing teams win a measley 2.3% of the time when they pull the goalie. Why is this, what is causing this discrepency? There are a few explanations, and the first is with the data itself. In our data, we used all third period goalie shifts that didn't last the whole period. This includes every time a team was losing at the end of the game, but it also included every time a goalie left the ice on a delayed penalty, and every time there was a mid-period goalie change. These inaccurate data points will make our findings less accurate, but it certainly doesn't explain the above discrepency. Another explanation is that there isn't parity in these games: if a team gets out to a big lead at the beginning of the game, then there's a good chance that they're simply a better team.

Next we'll look at how success changes as the defecit changes:

In [377]:
temp1 = pulled_goalies[(pulled_goalies["goal_diff"] == 1)]
temp2 = pulled_goalies[(pulled_goalies["goal_diff"] == 2)]
temp3 = pulled_goalies[(pulled_goalies["goal_diff"] == 3)]

print(temp1.loc[temp["didTheyComeBack"] == 1].shape[0]/temp1.shape[0])
print(temp2.loc[temp["didTheyComeBack"] == 1].shape[0]/temp2.shape[0])
print(temp3.loc[temp["didTheyComeBack"] == 1].shape[0]/temp3.shape[0])

0.1578708946772367
0.02038664323374341
0.009259259259259259


This indicates that pulling the goalie will help eliminate a 1, 2, and 3 goal defecit 15.79%, 2.04%, and .93% of the time, respectively. This tells us the obvious: the larger your opponent's lead, the harder it will be to claw your way back. Next we'll look at the times that winning and teams decide to take the goalie out:

In [383]:
temp1 = pulled_goalies
temp2 = pulled_goalies[pulled_goalies["didTheyComeBack"] == 1]
temp3 = pulled_goalies[pulled_goalies["didTheyComeBack"] == 0]

q1 = temp2["time_left"].quantile(.25)
q3 = temp2["time_left"].quantile(.75)
mask = temp2["time_left"].between(q1, q3, inclusive=True)
iqr_success = temp2.loc[mask, "time_left"]

q1 = temp3["time_left"].quantile(.25)
q3 = temp3["time_left"].quantile(.75)
mask = temp3["time_left"].between(q1, q3, inclusive=True)
iqr_fail = temp3.loc[mask, "time_left"]


print(temp1.loc[temp1["didTheyComeBack"] == 1, "time_left"].median())
print(temp1.loc[temp1["didTheyComeBack"] == 0, "time_left"].median())
print()
print(iqr_success.mean())
print(iqr_fail.mean())

121.0
116.0

155.90414507772022
121.37707997852925


The above block calculated the median time and interquartile mean time that losing teams were successful/unsuccessful when pulling the goalie: the median successful time is 121 seconds left, while the median unsuccessful time is 116 seconds. Meanwhile, the IQM successful time is a full 34 seconds sooner on average than the IQM unsuccessful time. This gives the indication that teams should pull their goalies sooner than they do now.

NOTE: I decided to use the interquartile mean instead of the regular mean in order to account for the very large and very small outliers in the dataset; the IQM is insensitive to outliers, unlike the regular mean.

Lastly, we'll attempt to model the relationship between the different independent variables with success rate. We'll start by creating a neural network that takes 3 inputs (time left in the game, the losing team's GPG, and the winning team's GAA) and returns one output, which we can interpret as the probability that a team will come back if a goalie is pulled in a given situation.

In [466]:

data = pulled_goalies[pulled_goalies["goal_diff"] == 1]

X = data[["time_left", "losingGPG", "winningGAA"]]
y = data["didTheyComeBack"]

reg = MLPRegressor(hidden_layer_sizes=(10,10), activation = 'logistic', max_iter=1000)
reg.fit(X, y)

# sit = [[time left, GPG, GAA]]
sit1 = [[60, 3, 3]]
sit2 = [[150, 2, 4]]
sit3 = [[300, 2, 4]]
sit4 = [[1200, 4, 3]]
print(reg.predict(sit1))
print(reg.predict(sit2))
print(reg.predict(sit3))
print(reg.predict(sit4))


[0.08567689]
[0.16666818]
[0.28829234]
[0.36887065]


There is a very clear problem with this model, and that's that it's very close to being proportional to the the time left in the game. Situation 4 above is one in which the losing team is down by 1 and there are 20 minutes left in the game; it's logically impossible for the losing team to have such a high chance of winning if they pull the goalie. Problems show up in other kinds of models as well:

In [473]:
data = pulled_goalies[pulled_goalies["goal_diff"] == 1]

X = data[["time_left", "losingGPG", "winningGAA"]]
y = data["didTheyComeBack"]

reg = SVR()
reg.fit(X, y)

# sit = [[time left, GPG, GAA]]
sit1 = [[60, 3, 3]]
sit2 = [[150, 3, 3]]
sit3 = [[300, 3, 3]]
sit4 = [[1200, 3, 3]]
print(reg.predict(sit1))
print(reg.predict(sit2))
print(reg.predict(sit3))
print(reg.predict(sit4))

[0.10004893]
[0.10033785]
[0.09958022]
[0.10627567]


This time we trained a support vector machine, and the numbers are practically indistinguishable from each other. Despite all of the data wrangling we did to get to this point, there is very little insight that we can gather from these machine learning models: the situation is simply too complicated to be predicted with the data points that we chose. So if we can't build a machine learning model that tells us when the optimal time to pull a goalie, what insights can we gain from this?

In [474]:
pulled_goalies.head()

,game_id,player_id,period,shift_start,shift_end,season,time_left,losingScore,losingTeam,winningScore,...,goalsForLosingTeam,didWinningTeamScore,goal_diff,losingGPG,winningGAA,success,didTheyWin,did_they_come_back,didTheyComeBack,net_goals
1599,2018020003,8469608,3,2400,3469.0,20182019,131.0,2,20,4,...,0,1,2,3.488372,2.908813,0,0,0,0,-1
2646,2018020004,8474889,3,2400,3511.0,20182019,89.0,2,28,4,...,0,1,2,3.470000,2.843036,0,0,0,0,-1
6597,2018020005,8475622,3,2400,3297.0,20182019,303.0,0,7,3,...,0,1,3,2.756098,2.324237,0,0,0,0,-1
8294,2018020008,8475852,3,2400,3418.0,20182019,182.0,0,12,1,...,1,0,1,2.897959,2.185846,0,0,1,1,1
8708,2018020006,8468685,3,2400,3489.0,20182019,111.0,1,3,2,...,0,1,1,2.768293,2.517451,0,0,0,0,-1


In [477]:
temp = pulled_goalies#[pulled_goalies["goal_diff"] == 1]



print(temp.shape[0])
print(temp["didWinningTeamScore"].sum())
print(temp["goalsForLosingTeam"].sum())
#print(temp.loc[(temp["goalsForLosingTeam"] == 0)&(temp["didWinningTeamScore"] == 0)])




8182
3232
1595
